<a href="https://colab.research.google.com/github/sayakpaul/Multimodal-Entailment-Baseline/blob/main/multimodal_entailment_attn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup and data collection

In [ ]:
!pip install -q tensorflow_text

In [ ]:
!wget -q https://github.com/sayakpaul/Multimodal-Entailment-Baseline/releases/download/v1.0.0/tweet_images.tar.gz
!tar xf tweet_images.tar.gz

## Imports

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow import keras

tf.random.set_seed(13)
np.random.seed(13)

## Data reading

In [ ]:
train_df = pd.read_csv("https://github.com/sayakpaul/Multimodal-Entailment-Baseline/raw/main/csvs/train_df.csv")
val_df = pd.read_csv("https://github.com/sayakpaul/Multimodal-Entailment-Baseline/raw/main/csvs/val_df.csv")
test_df = pd.read_csv("https://github.com/sayakpaul/Multimodal-Entailment-Baseline/raw/main/csvs/test_df.csv")

print(f"Total training examples: {len(train_df)}")
print(f"Total validation examples: {len(val_df)}")
print(f"Total test examples: {len(test_df)}")

Total training examples: 1197
Total validation examples: 63
Total test examples: 140


## Data input pipeline

In [ ]:
# Define TF Hub paths to the BERT encoder and its preprocessor.
bert_model_path = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1"
bert_preprocess_path = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

In [ ]:
# Reference:
# https://www.tensorflow.org/text/tutorials/bert_glue

def make_bert_preprocess_model(sentence_features, seq_length=128):
  """Returns Model mapping string features to BERT inputs.

  Args:
    sentence_features: a list with the names of string-valued features.
    seq_length: an integer that defines the sequence length of BERT inputs.

  Returns:
    A Keras Model that can be called on a list or dict of string Tensors
    (with the order or names, resp., given by sentence_features) and
    returns a dict of tensors for input to BERT.
  """

  input_segments = [
      tf.keras.layers.Input(shape=(), dtype=tf.string, name=ft)
      for ft in sentence_features]

  # Tokenize the text to word pieces.
  bert_preprocess = hub.load(bert_preprocess_path)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in input_segments]

  # Optional: Trim segments in a smart way to fit seq_length.
  # Simple cases (like this example) can skip this step and let
  # the next step apply a default truncation to approximately equal lengths.
  truncated_segments = segments

  # Pack inputs. The details (start/end token ids, dict of output tensors)
  # are model-dependent, so this gets loaded from the SavedModel.
  packer = hub.KerasLayer(bert_preprocess.bert_pack_inputs,
                          arguments=dict(seq_length=seq_length),
                          name='packer')
  model_inputs = packer(truncated_segments)
  return keras.Model(input_segments, model_inputs)

In [ ]:
bert_preprocess_model = make_bert_preprocess_model(['text_1', 'text_2'])

In [ ]:
# Reference:
# https://keras.io/examples/structured_data/structured_data_classification_from_scratch/
def dataframe_to_dataset(dataframe):
    columns = ["image_1_path", "image_2_path", "text_1", "text_2", "label_idx"]
    dataframe = dataframe[columns].copy()
    labels = dataframe.pop("label_idx")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    return ds

In [ ]:
resize = (128, 128)
bert_input_features = ['input_word_ids', 'input_type_ids', 'input_mask']

def read_resize(image_path):
    extension = tf.strings.split(image_path)[-1]

    image = tf.io.read_file(image_path)
    if extension == b"jpg":
        image = tf.image.decode_jpeg(image, 3)
    else:
        image = tf.image.decode_png(image, 3)
    image = tf.image.resize(image, resize)
    return image

def preprocess_text(text_1, text_2):
    text_1 = tf.convert_to_tensor([text_1])
    text_2 = tf.convert_to_tensor([text_2])
    output = bert_preprocess_model([text_1, text_2])
    output = {feature: tf.squeeze(output[feature]) 
        for feature in bert_input_features}
    return output

def preprocess(sample):
    image_1 = read_resize(sample["image_1_path"])
    image_2 = read_resize(sample["image_2_path"])
    text = preprocess_text(sample["text_1"], sample["text_2"])
    return {"image_1": image_1, "image_2": image_2, "text": text}

In [ ]:
batch_size = 32
auto = tf.data.AUTOTUNE

def prepare_dataset(df, training=True):
    ds = dataframe_to_dataset(df)
    if training:
        ds = ds.shuffle(len(train_df))
    ds = ds.map(lambda x, y: (preprocess(x), y))
    ds = ds.batch(batch_size).prefetch(auto)
    return ds

## Final datasets

In [ ]:
train_ds = prepare_dataset(train_df)
validation_ds = prepare_dataset(val_df, False)
test_ds = prepare_dataset(test_df, False)

In [ ]:
# Separate the train and test labels for later evaluation.
def separate_labels(ds):
    labels = []
    for _, label in ds.unbatch():
        labels.append(label)
    labels = np.array(labels)
    return labels

train_labels = separate_labels(train_ds)
test_labels = separate_labels(test_ds)

## Model utilities

`project_embeddings()`, `create_vision_encoder()`, and `create_text_encoder()` come from [here](https://keras.io/examples/nlp/nl_image_search/). 



In [ ]:
def project_embeddings(
    embeddings, num_projection_layers, projection_dims, dropout_rate
):
    projected_embeddings = keras.layers.Dense(units=projection_dims)(embeddings)
    for _ in range(num_projection_layers):
        x = tf.nn.gelu(projected_embeddings)
        x = keras.layers.Dense(projection_dims)(x)
        x = keras.layers.Dropout(dropout_rate)(x)
        x = keras.layers.Add()([projected_embeddings, x])
        projected_embeddings = keras.layers.LayerNormalization()(x)
    return projected_embeddings

In [ ]:
def create_vision_encoder(
    num_projection_layers, projection_dims, dropout_rate, trainable=False
):
    # Load the pre-trained ResNet50V2 model to be used as the base encoder.
    resnet_v2 = keras.applications.ResNet50V2(
        include_top=False, weights="imagenet", pooling="avg"
    )
    # Set the trainability of the base encoder.
    for layer in resnet_v2.layers:
        layer.trainable = trainable
    
    # Receive the images as inputs.
    image_1 = keras.Input(shape=(128, 128, 3), name="image_1")
    image_2 = keras.Input(shape=(128, 128, 3), name="image_2")
    
    # Preprocess the input image.
    preprocessed_1 = keras.applications.resnet_v2.preprocess_input(image_1)
    preprocessed_2 = keras.applications.resnet_v2.preprocess_input(image_2)
    
    # Generate the embeddings for the images using the resnet_v2 model
    # concatenate them.
    embeddings_1 = resnet_v2(preprocessed_1)
    embeddings_2 = resnet_v2(preprocessed_2)
    embeddings = keras.layers.Concatenate()([embeddings_1, embeddings_2])
    
    # Project the embeddings produced by the model.
    outputs = project_embeddings(
        embeddings, num_projection_layers, projection_dims, dropout_rate
    )
    # Create the vision encoder model.
    return keras.Model([image_1, image_2], outputs, name="vision_encoder")

In [ ]:
def create_text_encoder(
    num_projection_layers, projection_dims, dropout_rate, trainable=False
):
    # Load the pre-trained BERT model to be used as the base encoder.
    bert = hub.KerasLayer(
        bert_model_path,
        name="bert",
    )
    # Set the trainability of the base encoder.
    bert.trainable = trainable
    
    # Receive the text as inputs.
    bert_input_features = ['input_type_ids', 'input_mask', 'input_word_ids']
    inputs = {
        feature: keras.Input(shape=(128, ), dtype=tf.int32, name=feature)
        for feature in bert_input_features
    }
    
    # Generate embeddings for the preprocessed text using the BERT model.
    embeddings = bert(inputs)["pooled_output"]
    
    # Project the embeddings produced by the model.
    outputs = project_embeddings(
        embeddings, num_projection_layers, projection_dims, dropout_rate
    )
    # Create the text encoder model.
    return keras.Model(inputs, outputs, name="text_encoder")

In [ ]:
def create_multimodal_model(num_projection_layers=1, projection_dims=256, dropout_rate=0.1, 
                     vision_trainable=False, text_trainable=False, attention=False):
    # Receive the images as inputs.
    image_1 = keras.Input(shape=(128, 128, 3), name="image_1")
    image_2 = keras.Input(shape=(128, 128, 3), name="image_2")

    # Receive the text as inputs.
    bert_input_features = ['input_type_ids', 'input_mask', 'input_word_ids']
    text_inputs = {
        feature: keras.Input(shape=(128, ), dtype=tf.int32, name=feature)
        for feature in bert_input_features
    }

    # Create the encoders.
    vision_encoder = create_vision_encoder(num_projection_layers, projection_dims, dropout_rate, vision_trainable)
    text_encoder = create_text_encoder(num_projection_layers, projection_dims, dropout_rate, text_trainable)

    # Fetch the embedding projections.
    vision_projections = vision_encoder([image_1, image_2])
    text_projections = text_encoder(text_inputs)

    # Cross-attention.
    if attention:
        query_value_attention_seq = keras.layers.Attention(use_scale=True, dropout=0.2)(
            [vision_projections, text_projections]
        )

    # Concatenate the projections and pass through the classification layer.
    concatenated = keras.layers.Concatenate()([vision_projections, text_projections])
    if attention:
        concatenated = keras.layers.Concatenate()([concatenated, query_value_attention_seq])
    outputs = keras.layers.Dense(3, activation="softmax")(concatenated)
    return keras.Model([image_1, image_2, text_inputs], outputs)

## Model with cross-attention

In [ ]:
multimodal_model = create_multimodal_model(attention=True)
multimodal_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics="accuracy")
history = multimodal_model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=10
)

Epoch 1/10
38/38 [==============================] - 25s 454ms/step - loss: 0.8467 - accuracy: 0.8271 - val_loss: 0.6523 - val_accuracy: 0.8095
Epoch 2/10
38/38 [==============================] - 16s 405ms/step - loss: 0.4565 - accuracy: 0.8755 - val_loss: 0.5520 - val_accuracy: 0.8571
Epoch 3/10
38/38 [==============================] - 15s 394ms/step - loss: 0.3701 - accuracy: 0.8814 - val_loss: 0.5522 - val_accuracy: 0.8571
Epoch 4/10
38/38 [==============================] - 15s 398ms/step - loss: 0.3023 - accuracy: 0.8981 - val_loss: 0.7031 - val_accuracy: 0.8571
Epoch 5/10
38/38 [==============================] - 16s 424ms/step - loss: 0.1828 - accuracy: 0.9323 - val_loss: 0.8307 - val_accuracy: 0.8254
Epoch 6/10
38/38 [==============================] - 16s 407ms/step - loss: 0.1377 - accuracy: 0.9524 - val_loss: 0.8676 - val_accuracy: 0.8730
Epoch 7/10
38/38 [==============================] - 15s 398ms/step - loss: 0.0761 - accuracy: 0.9774 - val_loss: 1.0029 - val_accuracy: 0.8571

In [ ]:
_, acc = multimodal_model.evaluate(test_ds)
print(f"Accuracy on the test set: {round(acc * 100, 2)}%.")

5/5 [==============================] - 3s 394ms/step - loss: 1.0132 - accuracy: 0.7929
Accuracy on the test set: 79.29%.


## Model without cross-attention

In [ ]:
multimodal_model = create_multimodal_model(attention=False)
multimodal_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics="accuracy")
history = multimodal_model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=10
)

Epoch 1/10
38/38 [==============================] - 23s 434ms/step - loss: 1.1289 - accuracy: 0.8087 - val_loss: 0.6368 - val_accuracy: 0.8571
Epoch 2/10
38/38 [==============================] - 16s 420ms/step - loss: 0.4110 - accuracy: 0.8822 - val_loss: 0.6254 - val_accuracy: 0.8095
Epoch 3/10
38/38 [==============================] - 16s 407ms/step - loss: 0.3799 - accuracy: 0.8797 - val_loss: 0.6159 - val_accuracy: 0.8571
Epoch 4/10
38/38 [==============================] - 15s 393ms/step - loss: 0.3216 - accuracy: 0.9023 - val_loss: 0.5869 - val_accuracy: 0.8571
Epoch 5/10
38/38 [==============================] - 15s 398ms/step - loss: 0.2180 - accuracy: 0.9223 - val_loss: 0.9261 - val_accuracy: 0.8571
Epoch 6/10
38/38 [==============================] - 16s 424ms/step - loss: 0.1514 - accuracy: 0.9465 - val_loss: 0.9612 - val_accuracy: 0.8571
Epoch 7/10
38/38 [==============================] - 16s 411ms/step - loss: 0.1229 - accuracy: 0.9574 - val_loss: 0.9977 - val_accuracy: 0.8095

In [ ]:
_, acc = multimodal_model.evaluate(test_ds)
print(f"Accuracy on the test set: {round(acc * 100, 2)}%.")

5/5 [==============================] - 2s 316ms/step - loss: 1.3787 - accuracy: 0.7786
Accuracy on the test set: 77.86%.
